### Data Analysis template

#### Goal: Analyse time series data for number of people with a tertiary education in unisc adjacent areas

Sunshine coast SA4 - 316
MB North SA4 - 313
MB South SA4 - 314
Wide Bay SA4 - 319

In [1]:
c_year <- "2021"
c_pack <- "TSP"
c_geo <- "SA4"
c_area <- "QLD"

directory <- paste("Census", c_year, c_pack, c_geo, c_area, sep = "_")

In [2]:
pkgLoad <- function( packages = "required" ) {

    if( length( packages ) == 1L && packages == "required" ) {
        packages <- c( "readxl", "tidyverse", "openssl", "dplyr", "data.table",
                       "here", "httr","purrr"
        )
    }

    packagecheck <- match( packages, utils::installed.packages()[,1] )

    packagestoinstall <- packages[ is.na( packagecheck ) ]

    if( length( packagestoinstall ) > 0L ) {
        utils::install.packages( packagestoinstall,
                             repos = "http://cran.csiro.au"
        )
    } else {
        print( "All requested packages already installed" )
    }

    for( package in packages ) {
        suppressPackageStartupMessages(
            library( package, character.only = TRUE, quietly = TRUE )
        )
    }

}

In [3]:
unzip_file <- function(zip_path, dest_path){
  unzip(zip_path, exdir = dest_path)
}

In [4]:
download_file <- function(url, dest_path, extract_path, file_name){
  # Ensure the destination directory exists
  if (!dir.exists(dest_path)) {
    dir.create(dest_path, recursive = TRUE)
  }
  
  full_dest_path <- file.path(dest_path, file_name)
  # Adjust extract_path to not append file_name directly
  GET(url, write_disk(full_dest_path, overwrite = TRUE))
  
  if (grepl(".zip$", full_dest_path)){
    unzip_file(full_dest_path, extract_path)
    file.remove(full_dest_path)
  }
}

In [5]:
download_census_data <- function(c_year, c_pack, c_geo, c_area, dest_path, extract_path){
  url <- paste0("https://www.abs.gov.au/census/find-census-data/datapacks/download/", c_year, "_", c_pack, "_", c_geo, "_for_", c_area, "_short-header.zip")
  file_name <- paste0(c_year, "_", c_pack, "_", c_geo, "_for_", c_area, "_short-header.zip")
  download_file(url, dest_path, extract_path, file_name)
  files <- list.files(extract_path, full.names = TRUE) # Get full paths
  print("Success")
}

In [6]:
#read from excel function
url_from_excel <- function(excel_path, dest_path, extract_path){
  urls_df <- read_excel(excel_path, skip = 1, col_names = c("url", "file_name"))
  for (i in 1:nrow(urls_df)) {
    download_file(urls_df$url[i], dest_path, extract_path, urls_df$file_name[i])
  }
}

In [7]:
retrieve_table <- function(search_value, table_index, datafiles) {
    # Search for the index for the dynamic value
    index <- grep(search_value, table_index$V1, fixed = TRUE, ignore.case = FALSE)
    file_paths <- file.path(datafiles, table_index$V1[index])

    # Read and combine all CSVs that match
    combined_df <- map_dfr(file_paths, read.csv)
    
    
    # Return the combined dataframe
    return(combined_df)
}

In [8]:
pkgLoad()

[1] "All requested packages already installed"


In [9]:
base_path <- here(directory)
dest_path <- here(directory, "Download")
extract_path <- here(directory, "Input")
output_path <- here(directory, "Output")
if (!dir.exists(base_path)) {
  dir.create(output_path, recursive = TRUE)
}
# Create directories if they do not exist
paths <- list(dest_path, extract_path, output_path)
lapply(paths, function(path) {
  if (!dir.exists(path)) {
    dir.create(path, recursive = TRUE)
  }
})
download_census_data(c_year, c_pack, c_geo, c_area, dest_path, extract_path)
extract_path <- here(directory, "Input", "Metadata")
files <- list.files(extract_path, full.names = TRUE) # Get full paths
geog_def_source <- files[grepl("geog", files)]
data_def_source <- files[grepl("DataPack", files)]

#read each table in the excel file and create dataframe based on tablename
read_excel_sheets <- function(file) {
  tables <- excel_sheets(file)
  dataframes <- list()
  for (table in tables) {
    dataframes[[table]] <- read_excel(file, sheet = table)
  }
  return(dataframes)
}
setColumns <- function(df) {
  for (col in 1:ncol(df)) {
    # Find the first non-NA value in the column
    firstNonNAIndex <- which(!is.na(df[[col]]))[1]
    
    # Check if there is a non-NA value
    if (!is.na(firstNonNAIndex)) {
      # Set the column name to the first non-NA value
      colnames(df)[col] <- as.character(df[firstNonNAIndex, col])
    }
  }
  return(df)
}

geog_def <- read_excel_sheets(geog_def_source)
data_def <- read_excel_sheets(data_def_source)

split_data_frames <- function(df_list) {
  split_dfs <- list()
  
  for (name in names(df_list)) {
    df <- df_list[[name]]
    print(paste("Processing data frame:", name))
    
    # Ensure df is a data.table
    setDT(df)
    
    # Split the data frame by the first column (assuming it's the category column)
    split_list <- split(df, df[[1]])
    
    for (category_name in names(split_list)) {
      cat_df <- split_list[[category_name]]
      
      # Check if this category already exists in split_dfs
      if (category_name %in% names(split_dfs)) {
        # If so, Combine existing and new data frames, then remove duplicates
        combined_df <- rbind(split_dfs[[category_name]], cat_df)
        # Ensure the combined data frame is unique
        unique_combined_df <- unique(combined_df)
        split_dfs[[category_name]] <- unique_combined_df
      } else {
        #if this is first occurence of category, add it to the list
        split_dfs[[category_name]] <- cat_df
      }
    }
  }
  
  return(split_dfs)
}

split_list_of_dfs <- split_data_frames(geog_def)

#process data definitions 
data_def_process <- function(data_def) {
  #process table 1
  data_def[[1]] <- data_def[[1]][!is.na(data_def[[1]][,2]),]
  data_def[[1]] <- setColumns(data_def[[1]])
  data_def[[1]] <- data_def[[1]][-1,]
  #process table 2
  data_def[[2]] <- data_def[[2]][!is.na(data_def[[2]][,2]),]
  data_def[[2]] <- setColumns(data_def[[2]])
  data_def[[2]] <- data_def[[2]][-1,]
  return(data_def)
}
data_def <- data_def_process(data_def)

#export data def as two seperate csvs

output_path <- here(directory, "Output", "Index") 
if (!dir.exists(output_path)) {
  dir.create(output_path, recursive = TRUE)
}

write.csv(data_def[1], here(output_path, "data_def_tables.csv"), row.names = FALSE)
write.csv(data_def[2], here(output_path, "data_def_columns.csv"), row.names = FALSE)
meta_data <-  here(directory, "Output", "Index")
output_path <- here(directory, "Output", "ASGS") 
if (!dir.exists(output_path)) {
  dir.create(output_path, recursive = TRUE)
}
for (name in names(split_list_of_dfs)) {
  df <- split_list_of_dfs[[name]]
  output_path
  write.csv(df, file.path(output_path, paste0(name, ".csv")), row.names = FALSE)
}
#read each csv and create a column called AGSS_Key
files <- list.files(output_path, pattern = "\\.csv$", full.names = TRUE)
for (file in files) {
  df <- fread(file)
  df$AGSS_Key <- file
  #add value to the AGSS_key which is the AGSS_Code_2021 
  df$AGSS_Key <- openssl::md5(as.character(df$AGSS_Code_2021))
  write.csv(df, file, row.names = FALSE)
}

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[1] "Success"


New names:
• `` -> `...2`
• `` -> `...3`
New names:
• `` -> `...2`
• `` -> `...3`
• `` -> `...4`
• `` -> `...5`
• `` -> `...6`


[1] "Processing data frame: 2021_ASGS_MAIN_Structures"
[1] "Processing data frame: 2021_ASGS_GCCSA_Structure"
[1] "Processing data frame: 2021_ASGS_Indigenous_Structures"
[1] "Processing data frame: 2021_ASGS_SOS_UCL_Structures"
[1] "Processing data frame: 2021_ASGS_SUA_Structures"
[1] "Processing data frame: 2021_ASGS_Non_ABS_Structures"


In [10]:
output_path
dest_path <- here(directory, "Input")
extract_path <- here(directory, "Output")

[1] "C:/Users/joshu/Census_2021_TSP_SA4_QLD/Output/ASGS"

In [11]:
files <- list.files(dest_path, full.names = TRUE)
print(files)

[1] "C:/Users/joshu/Census_2021_TSP_SA4_QLD/Input/2021 Census TSP Statistical Area 4 for QLD"
[2] "C:/Users/joshu/Census_2021_TSP_SA4_QLD/Input/Metadata"                                  
[3] "C:/Users/joshu/Census_2021_TSP_SA4_QLD/Input/Readme"                                    


In [12]:
dest_path
extract_path

[1] "C:/Users/joshu/Census_2021_TSP_SA4_QLD/Input"

[1] "C:/Users/joshu/Census_2021_TSP_SA4_QLD/Output"

In [13]:
datafiles <- files[1]

In [14]:
table_index <- as.data.table((list.files(datafiles)))
table_verbose <- read.csv(file.path(meta_data, "data_def_tables.csv"))
print(table_index)

                              V1
                          <char>
  1:  2021Census_T01_QLD_SA4.csv
  2:  2021Census_T02_QLD_SA4.csv
  3: 2021Census_T03A_QLD_SA4.csv
  4: 2021Census_T03B_QLD_SA4.csv
  5: 2021Census_T03C_QLD_SA4.csv
 ---                            
 97: 2021Census_T33F_QLD_SA4.csv
 98: 2021Census_T33G_QLD_SA4.csv
 99: 2021Census_T33H_QLD_SA4.csv
100:  2021Census_T34_QLD_SA4.csv
101:  2021Census_T35_QLD_SA4.csv


In [15]:
meta_data

[1] "C:/Users/joshu/Census_2021_TSP_SA4_QLD/Output/Index"

In [16]:
table_verbose

Table.number..name..population.Table.number,Table.number..name..population.Table.name,Table.number..name..population.Table.population
<chr>,<chr>,<chr>
T01,Selected Person Characteristics by sex,Persons
T02,Selected Medians and Averages,NA
T03,Age by Sex,Persons
T04,Registered Marital Status by Age by Sex,Persons aged 15 years and over
T05,Social Marital Status by Age by Sex,Persons aged 15 years and over
T06,Indigenous Status by Age by Sex,Persons (excludes overseas visitors)
T07,Number of Children Ever Born by Age of Parent,Females aged 15 years and over (excludes overseas visitors)
T08,Country of Birth of Person by Sex,Persons
T09,Ancestry by country of birth of parents,Persons (excludes overseas visitors)


In [17]:
df <- retrieve_table("T31",table_index, datafiles)


In [18]:
head(df)

,SA4_CODE_2021,C11_M_Post_Deg_15_19_yrs,C11_M_Post_Deg_20_24_yrs,C11_M_Post_Deg_25_34_yrs,C11_M_Post_Deg_35_44_yrs,C11_M_Post_Deg_45_54_yrs,C11_M_Post_Deg_55_64_yrs,C11_M_Post_Deg_65_74_yrs,C11_M_Post_Deg_75_84_yrs,C11_M_Post_Deg_85over,⋯,C21_P_Tot_15_19_yrs,C21_P_Tot_20_24_yrs,C21_P_Tot_25_34_yrs,C21_P_Tot_35_44_yrs,C21_P_Tot_45_54_yrs,C21_P_Tot_55_64_yrs,C21_P_Tot_65_74_yrs,C21_P_Tot_75_84_yrs,C21_P_Tot_85over,C21_P_Tot_Tot
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,301,0,12,239,521,608,423,188,58,22,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,302,0,30,706,795,619,433,160,70,7,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,303,0,150,2600,2207,1651,991,372,106,15,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,304,0,78,1138,1652,1735,1417,721,244,48,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,305,0,156,2291,2305,1844,1221,458,101,31,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,306,0,8,158,346,394,298,175,46,13,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [19]:
df <- df %>% filter(SA4_CODE_2021 %in% c(316, 313, 314, 319))
head(df)

,SA4_CODE_2021,C11_M_Post_Deg_15_19_yrs,C11_M_Post_Deg_20_24_yrs,C11_M_Post_Deg_25_34_yrs,C11_M_Post_Deg_35_44_yrs,C11_M_Post_Deg_45_54_yrs,C11_M_Post_Deg_55_64_yrs,C11_M_Post_Deg_65_74_yrs,C11_M_Post_Deg_75_84_yrs,C11_M_Post_Deg_85over,⋯,C21_P_Tot_15_19_yrs,C21_P_Tot_20_24_yrs,C21_P_Tot_25_34_yrs,C21_P_Tot_35_44_yrs,C21_P_Tot_45_54_yrs,C21_P_Tot_55_64_yrs,C21_P_Tot_65_74_yrs,C21_P_Tot_75_84_yrs,C21_P_Tot_85over,C21_P_Tot_Tot
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,313,0,0,110,202,224,202,145,35,10,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,314,0,3,226,529,466,335,107,29,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,316,0,14,194,516,601,644,478,153,16,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,319,0,0,90,198,237,245,143,43,13,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,313,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,314,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [20]:
search_long <- function(search_value, table_desc) {
  # Search for the index for the dynamic value in the second column
  index <- grep(search_value, table_desc[, 2], fixed = TRUE, ignore.case = FALSE)
  # Return the 2nd and 3rd columns of matching rows
  result <- table_desc[index, c(2, 3)]
  return(result)
}

search_short <- function(search_value, table_desc) {
  # Replace spaces with underscores in the search value
  search_value <- gsub(" ", "_", search_value)

  # Search for the index for the dynamic value
  index <- grep(search_value, table_desc[, 3], fixed = TRUE, ignore.case = FALSE)
  
  # Return the 2nd and 3rd columns of matching rows
  result <- table_desc[index, c(2, 3)]
  return(result)
}

In [23]:
table_desc <- read.csv(file.path(meta_data, "data_def_columns.csv"))
search_long("C21_P_Tot_15", table_desc)
search_short("Age 15", table_desc)

,Cell.descriptors.information.Short,Cell.descriptors.information.Long
,<chr>,<chr>
13773,C21_P_Tot_15_19_yrs,2021_Census_Persons_Total_Age_15_19_years
15123,C21_P_Tot_15_19,2021_Census_Persons_Total_Age_15_19_years


,Cell.descriptors.information.Short,Cell.descriptors.information.Long
,<chr>,<chr>
12793,C11_M_Post_Deg_15_19_yrs,2011_Census_Males_Postgraduate_Degree_Level_Age_15_19_years
12803,C11_M_GrDip_GrCe_15_19,2011_Census_Males_Graduate_Diploma_and_Graduate_Certificate_Level_Age_15_19_years
12813,C11_M_BD_15_19,2011_Census_Males_Bachelor_Degree_Level_Age_15_19_years
12823,C11_M_AdD_DL_15_19,2011_Census_Males_Advanced_Diploma_and_Diploma_Level_Age_15_19_years
12833,C11_M_CL_C_III_IV_15_19,2011_Census_Males_Certificate_Level_Certificate_III_and_IV_Level_Age_15_19_years
12843,C11_M_CL_C_I_II_15_19,2011_Census_Males_Certificate_Level_Certificate_I_and_II_Level_Age_15_19_years
12853,C11_M_CL_CL_nfd_15_19,2011_Census_Males_Certificate_Level_Certificate_Level_nfd_Age_15_19_years
12863,C11_M_CL_Tot_15_19,2011_Census_Males_Certificate_Level_Total_Age_15_19_years
12873,C11_M_LE_ID_15_19,2011_Census_Males_Level_of_education_inadequately_described_Age_15_19_years
